<a id='top'></a>

# CSCI3022 F21
# Homework 3: Stochastic Simulation
***

**Name**:Caleb Starkey

***

This assignment is due on Canvas (as .ipynb) and Gradescope (as a .pdf) by **MIDNIGHT on Mon 20 Sep**. Your solutions to theoretical questions should be done in Markdown directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- Any relevant data sets should be available on Canvas. To make life easier on the graders if they need to run your code, do not change the relative path names here. Instead, move the files around on your computer.
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Here is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked on Canvas on writing math in Markdown. **All** of your written commentary, justifications and mathematical work should be in Markdown.  I also recommend the [wikibook](https://en.wikibooks.org/wiki/LaTeX) for LaTex.
- Because you can technically evaluate notebook cells is a non-linear order, it's a good idea to do **Kernel $\rightarrow$ Restart & Run All** as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- 45 points of this assignment are in problems.  The remaining 5 are for neatness, style, and overall exposition of both code and text.
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Problem 3](#p3) |
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

[Back to top](#top)
<a/ id='p1'></a>

## (10 pts) Problem 1: A Short Simulation

One of the big payoffs of simulation is that it can let us answer some probability questions that are otherwise quite difficult.  We can instead just simulate the process and get approximate results based on simulation.  We consider a few simplified variants of "games," beginning with a "solitaire" game.  

You begin with a standard, randomly shuffled deck of 52 cards. You then start dealing cards face up, one at a time, into a pile. As you deal each card, you also speak aloud, in order, the 13 card faces in a standard deck: ace, two, three, etc. (When you get to king, you start over at ace.) You keep doing this until the rank of the card you deal matches the rank you speak aloud, in which case you lose. You win if you reach the end of the deck without any matches.

**Part A**:  What is the probability that you lose immediately, on the first card?

**Solution**: $\frac{1}{13}$Being that the first card would have to be an ace, and there are four aces in a deck.


**Part B**:  Set up a simulation to answer the prompt: what is the probability that you make it all the way through the deck without a match and win?

Note that your final result should be *close* to the value of $(1-p)^{52}$, where $p$ was your result in part **A**.  This would be the theoretical result if each card flip was independent.  Was your simulated result higher or lower?  Does this match your intuition?

In [2]:
#sims

def simFlip(deck, currentCard):
    choice = np.random.choice(deck)
    if (choice == currentCard):
        return True
    else:
        deck.remove(choice)

def simSolitaire():        
    callOut = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    cards = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    np.random.shuffle(cards)
    for i in callOut:
        if(simFlip(cards,i)):
            return(0)    
    return(1)

results = np.zeros(1500)
for i in range(1500):
    results[i] = simSolitaire()
print("probability that you make it through the deck = {}".format(results.sum()/1500))

probability that you make it through the deck = 0.017333333333333333


The probability is close to that predicted by the value of $(1-\frac{1}{13})^{52}$ (.015777), however after playing it a few times there seems to be a large amount of variance(I observerd as high as .01) in the final probability. 

[Back to top](#top)
<a/ id='p2'></a>
## (30 pts) Problem 2: A Longer Simulation
**
Suppose we're now playing another card game, but this one uses custom cards.  In this game, your goal is to develop resources and use them to play actions.  Your deck of exactly 60 cards can consist of two types of cards: Resources (**R**) and Actions (**A**).

You want to figure out how many resources and how many actions to put into the deck.  Rather than doing some tedious algebra and counting, you decide to simulate playing the game to see how often specific amounts of *R* and *A* cards give you good hands.

1) Each game, you start with a hand of 7 cards.  A hand is *playable* if it contains at least 2 **R** cards and at least 2 **A** cards.

2) During the game, you draw one card per turn from your remaining deck.  You still want to have a good balance of **R** and **A** cards as the game progresses.  A hand is *good* if it contains at least 5 **R** cards in the first 15 cards (including your starting hand) but no more than 8 **R** cards over this same window.

3) A hand is *excellent* if it contains exactly 6 **R** cards in the first 13 cards, and then the next 2 cards are **A**.

**Part A**: Write a function `play_game` that shuffles your deck of **R** and **A** cards.  `play_game` should take as input argument the number of **R** cards in the deck and should return 3 True or False outputs: one each for whether or not the resulting hand was *playable*, *good*, and/or *excellent.*

Test run your function on some trivial cases such as `play_game(0)` and `play_game(60)` to ensure that you return "False, False, False."

**Part B**: Our goal is to choose the input argument of `play_game(R)` that gives us the best hands.  Without doing any simulation, explain what a reasonable choice of **R** would be to maximize the chances of getting a *playable* hand.

In [3]:
def play_game(R):
    deck = np.repeat('R',R)
    A = int(60-R)
    deck = np.concatenate((deck,np.repeat('A',A)))
    hand = np.random.choice(deck,7,False)
    playable= False 
    good= False
    excellent= False
    currentA = 0
    if((hand=='A').astype(int).sum() >= 2 and (hand=='R').astype(int).sum()>=2):
        playable = True 
    
    hand=np.concatenate((hand,np.random.choice(deck,6,False)))
    if((hand=='R').astype(int).sum() == 6):
        currentA = (hand=='A').astype(int).sum()
        excellent = True
    
    hand=np.concatenate((hand,np.random.choice(deck,2,False)))
    if(excellent == True and currentA+2 !=(hand=='A').astype(int).sum()):
        excellent == False
    
    if((hand=='R').astype(int).sum()>=5 and (hand=='R').astype(int).sum()<8):
        good = True

    
    return playable, good, excellent
print('With 0 resources:', play_game(0))
print('With 60 resources:', play_game(60))

With 0 resources: (False, False, False)
With 60 resources: (False, False, False)


**Part C**: Set up a data frame with the following columns: 'R', 'playable', 'good', 'excellent'.

Then fill in 1000 rows of the data frame by running `play_game` when `R=30`.  With this, answer the following prompts:

**C.1**: What is the approximate probability of a *good* hand when $R=30$?  

**C.2**: What is the approximate probability of an *excellent* hand when $R=30$?

**C.3**: What is the probability of a hand that is both *playable* and *good* when $R=30$?

**C.4**: What is the probability of a hand that is *good* **given** that the hand was *playable* when $R=30$?


In [4]:
columns = ['R','playable', 'good', 'excellent']
data= []
for i in range(1000):
    data.append((30,)+play_game(30))
theGame = pd.DataFrame(data,columns = columns)

print("the approximate probability of a good hand when R=30 is {}".format(theGame['good'].sum()/1000))
print("the approximate probability of an excellent hand when R=30 is {}".format(theGame['excellent'].sum()/1000))
print("the probability of a hand that is both playable and good when R=30 is {}".format(theGame.loc[theGame['playable'], 'good'].sum()/1000))
print("the probability of a hand that is good given that the hand was playable when R=30 is {}".format(theGame['good'].sum()/theGame['playable'].sum()))

the approximate probability of a good hand when R=30 is 0.405
the approximate probability of an excellent hand when R=30 is 0.231
the probability of a hand that is both playable and good when R=30 is 0.377
the probability of a hand that is good given that the hand was playable when R=30 is 0.4581447963800905


**Part D**: Using your data frame in **part C**, add additional rows for a variety of choices of $R$.  You may choose to simulate all values of $R$ from $[0, 1, 2, \dots 60]$ or some subset therein, as long as you are able to answer:

**D1**: What is the value of $R$ that is most likely to return hands that are both *playable* and *good*?

**D2**: What is the value of $R$ that is most likely to return hands that are *excellent*?

**D3**: Create a line plot with `plt.plot` to support your answers.

In [5]:
newData = []
playableGood = []
excellent = []

for i in range(61):
    tempPlay = 0 
    tempExce = 0
    for j in range(100):
        results = play_game(i)
        newData.append((i,)+results)
        if(results[0] and results[1]):
            tempPlay+=1
        if(results[2]):
            tempExce+=1
    playableGood.append(tempPlay/100)
    excellent.append(tempExce/100)
df = pd.DataFrame(newData,columns = columns)

playableMax = max(playableGood)
excellentMax = max(excellent)
print("the value of  𝑅  that is most likely to return hands that are both playable and good is {}".format(playableGood.index(playableMax)))
print("What is the value of  𝑅  that is most likely to return hands that are excellent is {}".format(excellent.index(excellentMax)))

the value of  𝑅  that is most likely to return hands that are both playable and good is 27
What is the value of  𝑅  that is most likely to return hands that are excellent is 25


[Back to top](#top)
<a/ id='p3'></a>

## (5 pts) Problem 3 (Theory): It's probably Bayes
***


To get published in an academic journal, you have to prove something "interesting."  As a result, most academics begin their research by investigating hypotheses that, all else equal, are unlikely to be true.  Suppose each research project begins with a research *hypothesis* that has a 12% chance of being correct.

They then perform a study that satisfies the following two properties:

    1) The probability that they correctly *find* an important result given that their hypothesis is true is 40%
    2) The probability that they incorrectly *find* an important result given that their hypothesis is false is 5%

If they *find* an important result they are published.  What is the probability that their *hypothesis* was true, given that they were published?

.048 probability of having a true hypothesis and being published

.044 probability of having a false hypothesis and being published 

Thus there is a .092 total chance of being published with a .522 probability that their hypothesis was true given that they were published